In [11]:
def find_cycles_naive(func, start_value, max_iters):
    history = [start_value]
    for generation in range(1, 1 + max_iters):
        value = func(history[-1])
        if value in history:
            offset = history.index(value)
            return offset, generation - offset + 1
        else:
            history.append(value)
    return max_iters, None

In [43]:
def collatz_conjecture(n):
    return n//2 if n % 2 == 0 else 3*n + 1

In [49]:
find_cycles_naive(collatz_conjecture, 1023, 100)

(60, 3)

In [12]:
def find_cycles_dict(func, start_value, max_iters):
    history = {start_value: 0}
    value = start_value
    for generation in range(1, 1 + max_iters):
        value = func(value)
        if value in history:
            offset = history[value]
            return offset, generation - offset
        else:
            history[value] = generation
    return max_iters, None

In [61]:
find_cycles_dict(collatz_conjecture, 1023, 100)

(60, 3)

In [13]:
def find_cycles_floyd(func, start_value, max_iters):
    tortoise = func(start_value)
    hare = func(tortoise)
    while tortoise != hare:
        tortoise = func(tortoise)
        hare = func(func(hare))
    offset = 0
    tortoise = start_value
    while tortoise != hare:
        tortoise = func(tortoise)
        hare = func(hare)
        offset += 1
    length = 1
    hare = func(tortoise)
    while tortoise != hare:
        hare = func(hare)
        length += 1
    return offset, length

In [67]:
find_cycles_floyd(collatz_conjecture, 1023, 100)

(60, 3)

In [71]:
for n in range(1024):
    naive_cycle = find_cycles_naive(collatz_conjecture, n, 100_000)
    dict_cycle = find_cycles_dict(collatz_conjecture, n, 100_000)
    floyd_cycle = find_cycles_floyd(collatz_conjecture, n, 100_000)
    assert naive_cycle == dict_cycle
    assert dict_cycle == floyd_cycle

In [69]:
def show_series(func, start_value, max_iters):
    value = start_value
    for _ in range(max_iters):
        print(value)
        value = func(value)

In [76]:
%timeit find_cycles_naive(collatz_conjecture, 1023, 100_000)

17.2 µs ± 248 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [77]:
%timeit find_cycles_dict(collatz_conjecture, 1023, 100_000)

6.46 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [78]:
%timeit find_cycles_floyd(collatz_conjecture, 1023, 100_000)

15.7 µs ± 289 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [80]:
offsets = [find_cycles_dict(collatz_conjecture, n, 100_000)[0] for n in range(2**20 - 1)]

In [50]:
def generate_periodic_function(offset, length):
    def internal(n):
        if n < offset - 1:
            return n + 1
        return offset + ((n + 1 - offset) % length)
    return internal

In [51]:
f = generate_periodic_function(5, 3)

In [52]:
for n in range(20):
    print(f(n))

1
2
3
4
5
6
7
5
6
7
5
6
7
5
6
7
5
6
7
5


In [53]:
find_cycles_dict(f, 0, 100)

(5, 3)

In [54]:
for offset in range(10):
    for length in range(1, 5):
        f = generate_periodic_function(offset, length)
        assert (offset, length) == find_cycles_floyd(f, 0, 0)